In [1]:
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

#########################################這區是告訴程式那些可行的Xpath######################################################
#Xpath 可以多用contains的方式來寫，並用or增加邏輯覆蓋率，如果在102-104行都顯示找不到XX，就可以在這邊手動增加，提高找尋到內容的機率

category_list = ['RETAIL', 'ECONOMY', 'FINANCE','US ECONOMY','MARKETS','SPEND','ON THE MONEY',
                 'TRADING NATION','WORLD POLITICS','CHINA ECONOMY','US MARKETS']

category_element_possible_xpath_list=['//div//a[contains(@class, "header_title last breadcrumb") or contains(@class, "Banner-eyebrow") or contains(@class, "ArticleHeader")]',
                              '//*[@id="main-article-header"]/div[1]/a']

author_element_possible_xpath_list=['//div//a[contains(@rel, "author") or contains(@class, "Author-authorName")  or contains(@class, "authorName")]']

article_element_possible_xpath_list = [ '//*[@id="article_body"]//div//div//p',
                                        '//*div[contains(@itemprop,"articleBody")]//p',
                                        '//*[@id="article_body"]/div[1]/div//p']

###########################################################################################################################

#CNBC的TRENDING 只會列出五篇
max_number_article = 5
main_driver = webdriver.Chrome('C:\chromedriver.exe')  # 注意你們放CHROMEDRIVER的位置
#driver.implicitly_wait(10)
main_driver.get('https://www.cnbc.com/world/?region=world')
#假裝滑動到該位置，讓javascript自動生成 (這個方式無法在headless的chrome內完成)
main_driver.execute_script("window.scrollTo(0, document.body.scrollHeight/9)")
#等待javascript生成完成
time.sleep(2)
#直接用Xpath取得文章標題
article_title_list=[]
address_list=[]

#要注意瀏覽器不要動它!不要更改大小!不然javascript又會再升成不同的Xpath，這樣17行也要更改才行!
for x in range(max_number_article):
    tmp = main_driver.find_element_by_xpath('//*[@id="chartbeatWidget_102535788"]/div/ul/li['+str(x+1)+']/div/div/a')
    article_title_list.append(tmp)
    address_list.append(tmp.get_attribute('href'))

#注意我method內爬文的寫法，分成三部分，1找分類 2找作者 3找內文
def Retrive_CNBC_Content(address, caegtory_list):
    driver = webdriver.Chrome('C:\chromedriver.exe') 
    driver.implicitly_wait(14)
    driver.get(address)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/6)")
    
    category_logged=""
    author_logged=""
    article_content_logged=""
    
    category_content=""
    author_content=""
    article_content=""
    
    try:
        #找分類#######################################################################
        for xpath in category_element_possible_xpath_list: 
            category_elements_found_list = driver.find_elements_by_xpath(xpath)
            category_mached_list =[]
            if(len(category_elements_found_list)>0):
                category_logged = "  找到類別"
                category_content = [t.text for t in category_elements_found_list]
                category_content = "/".join(category_content)
                
                category_mached_list = [c.text for c in category_elements_found_list if c.text in category_list]
                if(len(category_mached_list)==0): 
                    category_logged += ",但不是我們要的文章類別"
                #else: print(" , ".join(category_ckecked_list))
                
                break;
            else: category_logged = "  找不到文章類別"
        
        #找作者#######################################################################
        for xpath in author_element_possible_xpath_list: 
            author_elements_found_list = driver.find_elements_by_xpath(xpath)
            if(len(author_elements_found_list)>0):
                author_logged = "  找到作者"
                author_content = [t.text for t in author_elements_found_list] # return strings
                author_content = "/".join(author_content)
                #print(',作者是: '+author_content)#把作者呈現出來
                break
            else: author_logged = "  找不到作者"
        
        #找內文#######################################################################
        for xpath in article_element_possible_xpath_list:          
            article_elements_found_list = driver.find_elements_by_xpath(xpath)
            if(len(article_elements_found_list)>0):
                article_content_logged = "  找到文章"
                article_content = [t.text for t in article_elements_found_list] # return strings
                article_content = "".join(article_content)
                #print(article_content)#把文章呈現出來
                #print(len(article_content))#對於一些特殊版面的文章找不到就會是0
                break
            else: article_content_logged = "  找不到文章"
        #rint(status_logged)
    except NoSuchElementException:  #這個例外是要處理另一種排版的畫面，RETAIL類別的文章會有兩種排版畫面，因此兩種Xpath
        print("出現找不到element的例外!")
    
    #印出結果方便分析，如果遇到找不到內容的網站，就要點進連結去看是不是
    print('搜尋狀況: ')
    print(category_logged)
    print(author_logged)
    print(article_content_logged)
    
    print('類別: '+category_content)
    print('作者: '+author_content)
    print('內容:' +article_content)
    print("="*130)
    driver.close()
    
for article_title, address in zip(article_title_list, address_list):
    print('文章網址: '+address)
    print('文章標題: '+article_title.text)
    time.sleep(5)#休息五秒不要讓對方網站發現你是機器人一直開她的網頁
    Retrive_CNBC_Content(address,category_list)
        
main_driver.close() 

文章網址: https://www.cnbc.com/2018/12/31/tesla-ceo-elon-musks-predictions-for-the-future.html
文章標題: 3 predictions for the future, according to billionaire tech titan Elon Musk
搜尋狀況: 
  找到類別,但不是我們要的文章類別
  找到作者
  找到文章
類別: POWER PLAYERS
作者: Catherine Clifford
內容:"I think it's important to have a future that is inspiring and appealing. I just think there have to be reasons that you get up in the morning and you want to live. Like, why do you want to live? What's the point? What inspires you? What do you love about the future?" so says billionaire Tesla CEO Elon Musk in his 2017 TED talk.For Musk, that means building a reality where humans live on multiple planets. "And if we're not out there, if the future does not include being out there among the stars and being a multiplanet species, I find that it's incredibly depressing if that's not the future that we're going to have," Musk says.And when Musk thinks about the future, he does so like an engineer — practically, carefully and with a flow 

搜尋狀況: 
  找到類別
  找到作者
  找到文章
類別: MARKETS
作者: Michael Sheetz/Robert Hum
內容:This volatile market year was full of historical milestones and the end of the year will be no different for a key stock index.For the first time ever, the S&P 500 will end the year with a loss after being positive for the first three quarters. The benchmark index was up 9 percent through the first three quarters of the year. Then the October sell-off began. The S&P 500 fell 7 percent in October and accelerated those losses this month, in what is likely to be the index's worst December performance since the Great Depression.Read More: Here are the best and worst performing stocks of this tough and volatile year for the marketThe S&P 500 is set to end 2018 down nearly 7 percent. The fourth-quarter sell-off flies in the face of history, as the last three months are typically the strongest time of the year for the markets.
文章網址: https://www.cnbc.com/2018/12/31/activision-blizzard-plans-to-fire-its-cfo-for-an-unspecif